Step 1: Load data...

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from subprocess import check_output

print(check_output(["ls", "../input"]).decode("utf8"))

data = pd.read_csv("../input/movie_metadata.csv")

Step 2: Prepare data...

In [ ]:
data_features_raw = data[["duration","budget", "facenumber_in_poster",
     "director_facebook_likes", "cast_total_facebook_likes",
     "actor_1_facebook_likes", "actor_2_facebook_likes", "actor_3_facebook_likes"]].copy().fillna(0)
data_labels_raw = data[["imdb_score"]].copy().fillna(0)  
split = 4000

data_features_train_raw = data_features_raw[:split]
data_labels_train_raw = data_labels_raw[:split]

data_features_test_raw = data_features_raw[split:]
data_labels_test_raw = data_labels_raw[split:]

print(data_features_train_raw.describe())
print(data_labels_train_raw.describe())

Step 3: Build Model...

In [ ]:
PERIODS = 10
STEPS_PER_PERIOD = 100
BATCH_SIZE = 100
LEARNING_RATE = 0.0001

def model_fn(features, targets, mode, params):
  targets_float = tf.cast(targets, tf.float64)
  with tf.name_scope("model"):
    fc1 = tf.contrib.layers.fully_connected(features, 1024, activation_fn=tf.tanh, scope="fc1")
    fc2 = tf.contrib.layers.fully_connected(fc1, 1024, activation_fn=tf.tanh, scope="fc2")
    fc3 = tf.contrib.layers.fully_connected(fc2, 256, activation_fn=tf.tanh, scope="fc3")
    predicted = tf.contrib.layers.fully_connected(fc3, 1, scope="prediction")
    predicted_int = tf.cast(predicted, tf.int64)

  with tf.name_scope("train"):
    loss = tf.reduce_mean(tf.abs(predicted - targets_float), name="loss")
    optimizer = tf.train.AdamOptimizer(LEARNING_RATE)
    optimize_op = tf.group(optimizer.minimize(loss), tf.assign_add(tf.contrib.framework.get_global_step(), 1))

    predictions = tf.reshape(predicted_int, [-1])
    predictions_dict = {"results": predictions}

  return tf.contrib.learn.ModelFnOps(mode=mode, predictions=predictions_dict, loss=loss, train_op=optimize_op)

model = tf.contrib.learn.Estimator(model_fn=model_fn)

Step 4: Train Model...

In [ ]:
  for period in range (0, PERIODS): 
    batch_features = data_features_train_raw.sample(BATCH_SIZE)
    batch_indices = batch_features.index.values.tolist()
    batch_labels = data_labels_train_raw.ix[batch_indices]
 
    model.fit(x=batch_features, y=batch_labels, steps=STEPS_PER_PERIOD)
    print(model.evaluate(x=batch_features, y=batch_labels, steps=1))

In [ ]:
print("Train")
print(model.evaluate(x=data_features_train_raw, y=data_labels_train_raw, steps=1))
print("Test")
print (model.evaluate(x=data_features_test_raw, y=data_labels_test_raw, steps=1))